### Import the needed packages

In [15]:
# package for using the the custom scripts 
import sys
sys.path.insert(0, "./../")
# package for handling files and directories
import os

import wandb
# a custom package for handling the data
import utilities as utils

from huggingface_hub import HfApi

from dotenv import load_dotenv

load_dotenv()
os.environ["WANDB_SILENT"] = "true"

### Global variable

In [3]:
path_full = os.environ.get("ORI_PATH")
path_dataset_ori = os.path.join(path_full, "datasets", "splitted", "original")
path_dataset_aug = os.path.join(path_full, "datasets", "splitted", "augmented")
path_model = os.path.join(path_full, "data", "model")
wandb_init_name = "implement callbacks"
hf_commit_message = "implement callbacks"
hyperparam = {
    "batch_size": 16,
    "img_size": 128,
    "shuffle": True,
    "model_type": "U-net",
    "metrics": ["accuracy", "MeanIoU", "Precision", "Recall"],
    "split_ratio": [7,1,2],
    "dataset_balance": True,
    "weighted_training": True,
}
ori_path = {
    "train": os.path.join(path_dataset_ori, "train"),
    "val": os.path.join(path_dataset_ori, "val"),
    "test": os.path.join(path_dataset_ori, "test"),
}
aug_path = {
    "train": os.path.join(path_dataset_aug, "train"),
    "val": os.path.join(path_dataset_aug, "val"),
    "test": os.path.join(path_dataset_aug, "test"),
}

### Import the images and mask

In [4]:
# get the original dataset
train_ori_img, train_ori_mask = utils.load_img_mask(ori_path["train"])
val_ori_img, val_ori_mask = utils.load_img_mask(ori_path["val"])
test_ori_img, test_ori_mask = utils.load_img_mask(ori_path["test"])

# get the augmented dataset
train_aug_img, train_aug_mask = utils.load_img_mask(aug_path["train"])
val_aug_img, val_aug_mask = utils.load_img_mask(aug_path["val"])
test_aug_img, test_aug_mask = utils.load_img_mask(aug_path["test"])

### Create the batched dataset

In [5]:
# create the tensor dataset for original dataset
trainset_ori = utils.create_dataset(train_ori_img, train_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_ori = utils.create_dataset(val_ori_img, val_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_ori = utils.create_dataset(test_ori_img, test_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])

# create the tensor dataset for augmented dataset
trainset_aug = utils.create_dataset(train_aug_img, train_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_aug = utils.create_dataset(val_aug_img, val_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_aug = utils.create_dataset(test_aug_img, test_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])

### Calculate label weight

### Train model

In [6]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_ori"
# train the model using original dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_ori, valset_ori, testset_ori,
                                                path_model, file_model_name,
                                                50)
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 40s 547ms/step - loss: 0.0092 - mean_px_acc: 0.5571 - auc: 0.7676 - precision: 0.6040 - recall: 0.4748 - val_loss: 0.0098 - val_mean_px_acc: 0.8050 - val_auc: 0.7999 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 10s 152ms/step - loss: 0.0065 - mean_px_acc: 0.6298 - auc: 0.8828 - precision: 0.7595 - recall: 0.6281 - val_loss: 0.0106 - val_mean_px_acc: 0.9829 - val_auc: 0.5822 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
16/16 [==============================] - 8s 142ms/step - loss: 0.0057 - mean_px_acc: 0.7164 - auc: 0.9123 - precision: 0.8053 - recall: 0.6565 - val_loss: 0.0112 - val_mean_px_acc: 0.9824 - val_auc: 0.5186 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
16/16 [==============================] - 9s 215ms/step - loss: 0.0053 - mean_px_acc: 0.8219 - auc: 0.9251 - precision: 0.8243 - recall: 0.6775 - val_loss: 0.0114 - val_mean_px_a

In [7]:
api = HfApi()
repo = api.create_repo("CAD-Glaucoma_Segmentation", repo_type="model", exist_ok=True)

In [9]:
hyperparam["aug"] = False
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"ori - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [10]:
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=hf_commit_message,
)

unet_model_ori.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 856kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/142f820abf9a0bb6822afd698c68f0e0e0e8b0bd', commit_message='implement callbacks', commit_description='', oid='142f820abf9a0bb6822afd698c68f0e0e0e8b0bd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)

In [11]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_aug"
# train the model using augmented dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_aug, valset_aug, testset_aug,
                                                path_model, file_model_name,
                                                50)
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 17s 328ms/step - loss: 0.0080 - mean_px_acc: 0.5645 - auc: 0.8314 - precision: 0.6950 - recall: 0.5947 - val_loss: 0.0095 - val_mean_px_acc: 0.6282 - val_auc: 0.7633 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 5s 119ms/step - loss: 0.0058 - mean_px_acc: 0.6154 - auc: 0.9087 - precision: 0.8008 - recall: 0.6827 - val_loss: 0.0099 - val_mean_px_acc: 0.9840 - val_auc: 0.7393 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
16/16 [==============================] - 5s 144ms/step - loss: 0.0053 - mean_px_acc: 0.7471 - auc: 0.9240 - precision: 0.8199 - recall: 0.6990 - val_loss: 0.0110 - val_mean_px_acc: 0.9827 - val_auc: 0.5228 - val_precision: 0.9330 - val_recall: 0.0773
Epoch 4/50
16/16 [==============================] - 6s 132ms/step - loss: 0.0049 - mean_px_acc: 0.8456 - auc: 0.9356 - precision: 0.8438 - recall: 0.7493 - val_loss: 0.0114 - val_mean_px_acc: 0.982

In [12]:
hyperparam["aug"] = True
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"aug - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [13]:
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=f"aug - {hf_commit_message}",
)

unet_model_aug.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 1.22MB/s]


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/2b0ea3c3a02db024c9e4977bab7e2e45c6bbd9ba', commit_message='aug - implement callbacks', commit_description='', oid='2b0ea3c3a02db024c9e4977bab7e2e45c6bbd9ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)